In [ ]:
import pandas as pd
import numpy as np
import editdistance

import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder



In [ ]:

kappa_matrix = pd.read_csv("./data/analysis/results/matrices/kappa_matrix.csv")
kappa_matrix.index = kappa_matrix.columns

coincidences_matrix = pd.read_csv("./data/analysis/results/matrices/coincidencias_matrix.csv")
coincidences_matrix.index = coincidences_matrix.columns

prompts = [
    "IA con experiencia en análisis de lenguaje y emociones, su tarea es analizar el sentimiento del siguiente texto. Considere el tono general de la discusión, la emoción transmitida por el lenguaje utilizado y el contexto en el que se utilizan las palabras y frases. Indique si el sentimiento es generalmente positivo, negativo o neutral y proporcione una etiqueta sin ningún texto adicional.",
    "la tarea de clasificación de sentimientos. Dado el texto, asigne una etiqueta de sentimiento entre positivo, negativo o neutral. Devuelva únicamente la etiqueta sin ningún otro texto.",
    "Indique si el sentimiento es generalmente positivo, negativo o neutral y proporcione una etiqueta sin ningún texto adicional. Considere el tono general de la discusión, la emoción transmitida por el lenguaje utilizado y el contexto en el que se utilizan las palabras y frases. Como IA con experiencia en análisis de lenguaje y emociones, su tarea es analizar el sentimiento del siguiente texto.",
    "únicamente la etiqueta sin ningún otro texto. Dado el texto, asigne una etiqueta de sentimiento entre positivo, negativo o neutral. Realice la tarea de clasificación de sentimientos. ",
    "IA experto en procesamiento de lenguaje natural y análisis de sentimientos, su tarea es analizar el sentimiento del siguiente texto. Considere el tono general del comentario, los sentimientos transmitidos por el texto utilizado y el contexto en el que se utilizan las palabras y oraciones. Indique si el sentimiento es generalmente positivo, negativo o neutral y proporcione una etiqueta sin ningún texto adicional.",
    "la tarea de clasificación de sentimientos. Dado el comentario, asigne una etiqueta de sentimiento entre positivo, negativo o neutral. Devuelva únicamente la etiqueta sin ningún otro texto.",
    "IA con experiencia en análisis de lenguaje y emociones, debes analizar el sentimiento del siguiente texto. Puedes considerar el tono general de la discusión, la emoción transmitida por el lenguaje utilizado y el contexto en el que se utilizan las palabras y frases. Tienes que indicar si el sentimiento es generalmente positivo, negativo o neutral y proporcione una etiqueta sin ningún texto adicional.",
    "realizar la tarea de clasificación de sentimientos. Dado el texto, puedes asignar una etiqueta de sentimiento entre positivo, negativo o neutral. Tienes que devolver únicamente la etiqueta sin ningún otro texto.",
    "IA experiencia análisis lenguaje emociones tarea analizar sentimiento siguiente texto Considere tono general  discusión emoción transmitida lenguaje utilizado contexto  utilizan palabras frases indique si sentimiento generalmente positivo negativo neutral proporcione etiqueta ningún texto adicional",
    "tarea clasificación sentimientos dado texto asigne etiqueta sentimiento entre positivo negativo neutral devuelva únicamente etiqueta",
]

In [ ]:
# Creates a formatted table with the kappa matrix and saves to excel
kappa_matrix.map(lambda x: "{:.2f}".format(x)).to_excel("data/analysis/results/kappa-matrix.xlsx")

In [ ]:
# Creates a formatted table with the coincidences matrix and saves to excel
coincidences_matrix.map(lambda x: "{:.2f}".format(x)).to_excel("data/analysis/results/coincidences-matrix.xlsx")

# Measures Levenshtein (Edit) Distance between prompts, but using tokens instead of characters

In [ ]:
# Tokenize the prompts
prompts_tokenized = [word_tokenize(prompt.lower()) for prompt in prompts]

# Convert tokens to numerical representation using LabelEncoder
all_tokens = [token for prompt in prompts_tokenized for token in prompt]
label_encoder = LabelEncoder()
label_encoder.fit(all_tokens)

prompts_numerical = [label_encoder.transform(prompt) for prompt in prompts_tokenized]

# Create an Edit Distance Matrix based on numerical prompts
num_prompts = len(prompts)
edit_distance_matrix = np.zeros((num_prompts, num_prompts))

# Calculate Edit distances between each pair of numerical prompts using editdistance library
for i in range(num_prompts):
    for j in range(num_prompts):
        if i != j:
            edit_distance_matrix[i][j] = editdistance.eval(
                prompts_numerical[i], prompts_numerical[j]
            )

# Convert the matrix to a DataFrame for easier handling
edit_distance_df = pd.DataFrame(
    edit_distance_matrix,
    columns=[f"prompt {i+1}" for i in range(num_prompts)],
    index=[f"prompt {i+1}" for i in range(num_prompts)],
)

# Normalize Cohen's Kappa and Edit Distance Matrices
kappa_matrix_normalized = (kappa_matrix - kappa_matrix.min().min()) / (
    kappa_matrix.max().max() - kappa_matrix.min().min()
)
edit_distance_df_normalized = (edit_distance_df - edit_distance_df.min().min()) / (
    edit_distance_df.max().max() - edit_distance_df.min().min()
)
edit_distance_df.map(lambda x: "{:.0f}".format(x)).to_excel(
    "data/analysis/results/levenshtein-matrix.xlsx"
)

# Correlations
- Between Kappa matrix and Levenshtein matrix
- Between coincidences matrix and Levenshtein matrix


In [ ]:
# Calculate the correlation between the normalized Cohen's Kappa matrix and the Edit Distance matrix
# Flatten the upper triangle of both matrices to get the values for correlation
kappa_values = kappa_matrix_normalized.where(
    np.triu(np.ones(kappa_matrix_normalized.shape), k=1).astype(bool)
).stack()
edit_distance_values = edit_distance_df_normalized.where(
    np.triu(np.ones(edit_distance_df_normalized.shape), k=1).astype(bool)
).stack()
coincidences_values = coincidences_matrix.where(
    np.triu(np.ones(coincidences_matrix.shape), k=1).astype(bool)
).stack()

# Calculate correlation
correlation = kappa_values.corr(edit_distance_values)
print(
    f"Correlation between Normalized Cohen's Kappa and Normalized Edit Distance: {correlation:.2f}"
)

# Calculate correlation
correlation = coincidences_values.corr(edit_distance_values)
print(
    f"Correlation between Normalized Coincidences and Normalized Edit Distance: {correlation:.2f}"
)

# Reports

In [ ]:
# Normalize Cohen's Kappa and Levenshtein Distance Matrices
kappa_matrix_normalized = (kappa_matrix - kappa_matrix.min().min()) / (
    kappa_matrix.max().max() - kappa_matrix.min().min()
)
coincidences_matrix_normalized = (
    coincidences_matrix - coincidences_matrix.min().min()
) / (coincidences_matrix.max().max() - coincidences_matrix.min().min())
edit_distance_df_normalized = 1 - (edit_distance_df - edit_distance_df.min().min()) / (
    edit_distance_df.max().max() - edit_distance_df.min().min()
)

# Visualization

# Heatmap for Cohen's Kappa Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(kappa_matrix_normalized, annot=True, cmap="Blues", cbar=True)
plt.title("Normalized Cohen's Kappa Matrix")
plt.savefig("data/analysis/results/heatmap-cohen-kappa.png")
plt.show()

# Heatmap for Coincidences Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(coincidences_matrix_normalized, annot=True, cmap="Greens", cbar=True)
plt.title("Normalized Coincidences Matrix")
plt.savefig("data/analysis/results/heatmap-coincidences.png")
plt.show()

# Heatmap for Levenshtein Distance Matrix without floating point format
plt.figure(figsize=(10, 8))
sns.heatmap(edit_distance_df_normalized, annot=True, cmap="Reds", cbar=True)
plt.title("1 - Normalized Levenshtein Distance Matrix")
plt.savefig("data/analysis/results/heatmap-levenshtein.png")
plt.show()